# Guardian - Iteration 14


## Import Packages

In [11]:
# load .env File in Environment
from dotenv import load_dotenv
# operating system interfaces - get Environment-variables
import os
# Connect to API
import requests
# create Pandas Data Frame
import pandas as pd
import datetime as dt
from functools import wraps
load_dotenv()

True

### Set Params

In [12]:
iteration = 14

content_url = "https://content.guardianapis.com/search"
all_content_params = {
    "page-size": "50",
    "page-size": "50",
    "show-fields": "all",
    "show-tags": "all",
    "from-date": "2016-11-01",
    "to-date": "2017-03-01",
    "order-by": "newest",
    "api-key": os.getenv("GUARDIAN_API_KEY4"),
}


In [13]:
def log_step(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        tic = dt.datetime.now()
        result = func(*args, **kwargs)
        time_taken = str(dt.datetime.now() - tic)
        print(f"{func.__name__}:\n shape={result.shape} took {time_taken}s\n")
        return result

    return wrapper

### Create DataFrame function


In [14]:
def get_df(url: str, params: dict):
        all_results = []
        current_page = 1
        total_pages = 1
        while current_page <= total_pages:
                tic = dt.datetime.now()
                params["page"] = current_page
                try:
                        r = requests.get(url, params)
                        all_results = all_results + r.json()["response"]["results"]
                        r.raise_for_status()
                except Exception as err:
                        SystemExit(err)
                if current_page == 1:
                        print("---- API STATUS ---- ")
                        print("status",  r.json()["response"]["status"])
                        total_pages = r.json()['response']['pages']
                        print("URL: ", r.url)
                        print("status",  r.json()["response"]["status"])
                        print("total",  r.json()["response"]["total"])
                        print("startIndex",  r.json()["response"]["startIndex"])
                        print("pageSize",  r.json()["response"]["pageSize"])
                        print("pages",  r.json()["response"]["pages"])
                        print("orderBy",  r.json()["response"]["orderBy"])
                        print("---- RUNTIME STATUS ---- ")

                time_taken = str(dt.datetime.now() - tic)
                print(f"({current_page}/{total_pages}) in {time_taken}s")
                        
                current_page += 1

        return pd.DataFrame(all_results)

### API Request

In [15]:
init_df = get_df(content_url, all_content_params)

---- API STATUS ---- 
status ok
URL:  https://content.guardianapis.com/search?page-size=50&show-fields=all&show-tags=all&from-date=2016-11-01&to-date=2017-03-01&order-by=newest&api-key=295c6540-8c34-4390-ad47-0cf94ebc153f&page=1
status ok
total 31368
startIndex 1
pageSize 50
pages 628
orderBy newest
---- RUNTIME STATUS ---- 
(1/628) in 0:00:01.390366s
(2/628) in 0:00:01.010883s
(3/628) in 0:00:01.135860s
(4/628) in 0:00:01.001081s
(5/628) in 0:00:01.336657s
(6/628) in 0:00:01.077727s
(7/628) in 0:00:00.880051s
(8/628) in 0:00:01.051683s
(9/628) in 0:00:01.204248s
(10/628) in 0:00:01.398523s
(11/628) in 0:00:06.008888s
(12/628) in 0:00:34.024857s
(13/628) in 0:00:01.379013s
(14/628) in 0:00:01.169397s
(15/628) in 0:00:02.037141s
(16/628) in 0:00:01.473524s
(17/628) in 0:00:00.952073s
(18/628) in 0:00:02.509078s
(19/628) in 0:00:15.354742s
(20/628) in 0:00:00.834744s
(21/628) in 0:00:01.540849s
(22/628) in 0:00:45.851112s
(23/628) in 0:00:14.587597s
(24/628) in 0:00:01.120975s
(25/628) i

In [16]:
@log_step
def init_pipeline(df):
    return df.copy()

@log_step
def unfold_columns(df):
    dict_cols = ["fields"] 
    for col in dict_cols:
        new_df = pd.DataFrame()
        new_df[col] = df[col]#.apply(lambda x: eval(x))
        add_cols_df = pd.json_normalize(new_df[col])
        df = pd.concat([df, add_cols_df], axis=1)
    
    # Tags extraction
    
    #df["tags"] = df["tags"].apply(lambda x: eval(x))
    df['tagWebTitle'] = df['tags'].map(lambda x:[i['webTitle'] for i in x])
    df['tagId'] = df['tags'].map(lambda x:[i['id'] for i in x])
    df = df.drop(columns="tags")
    return df.drop(columns=dict_cols)
@log_step
def drop_columns(df):
    df = df.drop(columns=[
        "body",
        "webUrl",
        "apiUrl",
        "sectionId",
        "pillarId",
        "showAffiliateLinks",
        "bylineHtml",
        "standfirst",
        "shouldHideReaderRevenue",
        "isInappropriateForSponsorship",
        "shortUrl",
        "isPremoderated",
        "charCount",
        "shouldHideAdverts",
        "showInRelatedContent",
        "lang",
        "main",
        "thumbnail"
        
    ])
    return df

@log_step
def create_csv(df):
    df.to_csv(f"../../data/partial/articles_partial_{iteration}_1.csv.zip", compression="zip")
    return df

In [17]:
articles = (
    init_df
        .pipe(init_pipeline)
        .pipe(unfold_columns)
        .pipe(drop_columns)
        .pipe(create_csv)
)

init_pipeline:
 shape=(31368, 13) took 0:00:00.010969s

unfold_columns:
 shape=(31368, 49) took 0:00:01.359619s

drop_columns:
 shape=(31368, 31) took 0:00:00.019529s

create_csv:
 shape=(31368, 31) took 0:00:20.340734s

